In [ ]:
import pandas as pd
from pangaeapy.pandataset import PanDataSet

pds = PanDataSet(897888)

header = {}
for k, event in enumerate(pds.events):
    header.update(
        {
            k: {
                "longitude": event.longitude,
                "latitude": event.latitude,
                "elevation": event.elevation,
                "event": event.label,
            },
        },
    )

df = pds.data
pdf = pd.DataFrame.from_dict(header).T

In [ ]:
cabo_frio = [
    "HAB_A01",
    "HAB_A02",
    "HAB_A03",
    "HAB_A04",
    "HAB_B1",
    "HAB_B2",
]


pdf = pdf[~pdf["event"].isin(cabo_frio)]
df = df[~df["Event"].isin(cabo_frio)]

In [ ]:
bbox = (
    pdf["longitude"].min(),
    pdf["longitude"].max(),
    pdf["latitude"].min(),
    pdf["latitude"].max(),
)

In [ ]:
from oceans.datasets import woa_subset

ds = woa_subset(*bbox, variable="temperature", time_period="annual", resolution="1/4")

ds = ds.sel(depth=0)

In [ ]:
%%time

ds.compute()

In [ ]:
woa = {}

for k, row in pdf.iterrows():
    tmp = ds.sel(
        {"lon": row["longitude"], "lat": row["latitude"]},
        method="nearest",
    )

    woa.update(
        {
            row["event"]: {
                "tmp": tmp["t_mn"].to_numpy().squeeze(),
                "lon": tmp["lon"].to_numpy(),
                "lat": tmp["lat"].to_numpy(),
                "lon_obs": row["longitude"],
                "lat_obs": row["latitude"],
            }
        }
    )

In [ ]:
import seawater as sw

w = pd.DataFrame(woa).T

lats = list(zip(w["lat"], w["lat_obs"]))
lons = list(zip(w["lon"], w["lon_obs"]))
dists = [sw.dist(*location)[0][0] for location in zip(lats, lons)]
w["dist"] = dists

In [ ]:
pd.concat([df[["SST (1-12)", "Event"]].reset_index(), w.reset_index()], axis=1)